In [1]:
from cub2011 import *
from torchvision.models import resnet18, ResNet18_Weights
from torchvision import transforms
import torch.nn as nn
import torch

# Define the transform with augmentation
transform = transforms.Compose([
    transforms.RandomResizedCrop(224, scale=(0.8, 1.0)),  # 随机裁剪并调整大小
    transforms.RandomHorizontalFlip(),                    # 随机水平翻转
    # transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),  # 随机颜色抖动
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Define the transform
# transform = transforms.Compose([
#     transforms.Resize((224, 224)),
#     transforms.ToTensor(),
#     transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
# ])

# load dataset
train = Cub2011("", transform=transform)
test = Cub2011("", train=False, transform=transform)

# see example data
# img, target = train.__getitem__(999)
# img.show()

# load resnet18
net = resnet18(weights=ResNet18_Weights.IMAGENET1K_V1)
# net = resnet18()
net.fc = nn.Linear(512, 200)

# move the net to GPU
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
net.to(device)


Files already downloaded and verified
Files already downloaded and verified


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [2]:
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.optim.lr_scheduler import StepLR
from torch.utils.tensorboard import SummaryWriter


# Loss
criterion = nn.CrossEntropyLoss()

# Separate parameters into two groups
base_lr = 0.001
finetune_rate = 0.01
finetune_lr = finetune_rate * base_lr
params_to_update = [
    {'params': net.conv1.parameters(), 'lr': finetune_lr},
    {'params': net.layer1.parameters(), 'lr': finetune_lr},
    {'params': net.layer2.parameters(), 'lr': finetune_lr},
    {'params': net.layer3.parameters(), 'lr': finetune_lr},
    {'params': net.layer4.parameters(), 'lr': finetune_lr},
    {'params': net.fc.parameters(), 'lr': base_lr}  # The last layer
]

# optimizer
optimizer = optim.SGD(params_to_update, momentum=0.9, weight_decay=1e-4)

# Learning rate sechuler
scheduler = StepLR(optimizer, step_size=50, gamma=0.1)

train_loader = DataLoader(train, batch_size=32, shuffle=True, num_workers=4)
test_loader = DataLoader(test, batch_size=32, shuffle=False, num_workers=4)

# TENSORBOARD
writer = SummaryWriter(log_dir='Resnet18/resnet18-lre-3-ftrate-2')

# training loop
num_epochs = 110
for epoch in range(num_epochs):
    # train
    net.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    train_loss = running_loss / len(train_loader)
    print(f"Epoch {epoch+1}/{num_epochs}, Training Loss: {train_loss:.3f}", end=", ")
    
    # Write training loss to TensorBoard
    writer.add_scalar('Training Loss', train_loss, epoch)

    # validation
    net.eval()
    correct = 0
    total = 0
    running_loss = 0.0
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)  # Move data to the device
            outputs = net(inputs)
            loss = criterion(outputs, labels)
            running_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    val_loss = running_loss / len(test_loader)
    accuracy = 100 * correct / total
    print(f'Validation Loss: {val_loss:.3f}', end=", ")
    print(f'Accuracy on test set: {accuracy:.3f}%')

    # Write validation loss and accuracy to TensorBoard
    writer.add_scalar('Validation Loss', val_loss, epoch)
    writer.add_scalar('Accuracy', accuracy, epoch)

    # Update the learning rate
    scheduler.step()
    
# Close the TensorBoard writer
writer.close()



Epoch 1/110, Training Loss: 5.246, Validation Loss: 4.925, Accuracy on test set: 5.661%
Epoch 2/110, Training Loss: 4.719, Validation Loss: 4.467, Accuracy on test set: 15.499%
Epoch 3/110, Training Loss: 4.275, Validation Loss: 4.083, Accuracy on test set: 24.560%
Epoch 4/110, Training Loss: 3.882, Validation Loss: 3.742, Accuracy on test set: 32.137%
Epoch 5/110, Training Loss: 3.544, Validation Loss: 3.472, Accuracy on test set: 35.261%
Epoch 6/110, Training Loss: 3.249, Validation Loss: 3.187, Accuracy on test set: 40.680%
Epoch 7/110, Training Loss: 2.975, Validation Loss: 2.986, Accuracy on test set: 43.303%
Epoch 8/110, Training Loss: 2.751, Validation Loss: 2.789, Accuracy on test set: 46.272%
Epoch 9/110, Training Loss: 2.551, Validation Loss: 2.643, Accuracy on test set: 47.687%
Epoch 10/110, Training Loss: 2.384, Validation Loss: 2.492, Accuracy on test set: 51.622%
Epoch 11/110, Training Loss: 2.226, Validation Loss: 2.369, Accuracy on test set: 52.295%
Epoch 12/110, Train

In [3]:
# !tensorboard --logdir=runs